In [4]:
import os, re
from typing_extensions import  List, Literal
import operator
from pydantic import BaseModel, Field
from typing_extensions import TypedDict

# LLM og verktøy

from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate
from llama_index.core import (get_response_synthesizer)


from llama_index.core import (StorageContext,  load_index_from_storage)
from llama_index.core.base.response.schema import Response
from llama_index.core.query_engine import BaseQueryEngine
from llama_index.core.schema import NodeWithScore

# Import av langchain og langgraph
from langchain_openai import AzureChatOpenAI
from langgraph.constants import Send
from langgraph.graph import StateGraph, START, END

# Indeksverktøy
LLMGPT4omini = AzureChatOpenAI(
    model=os.getenv('AZURE_OPENAI_MODEL_GPT4omini'),
    deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME_GPT4omini'),
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME_GPT4omini'),
    api_key=os.getenv('AZURE_OPENAI_API_KEY_GPT4omini'),
    azure_endpoint=os.getenv('AZURE_OPENAI_AZURE_ENDPOINT_GPT4omini'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSJON_GPT4omini'),
    temperature=0.0,
    timeout= 120,
)

def read_index_from_storage(storage):
    storage_context = StorageContext.from_defaults(persist_dir=storage)
    return load_index_from_storage(storage_context)

# Sett Azure OpenAI-legitimasjon

llm = LLMGPT4omini

chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "You are a helpful assistant, and you will be given a user request."+
            "\nYou will respond with empathy"+
            "\nYou will answer in language that young people aged 13 to 19 understand"+
            "\nSome rules to follow:"+
            "\n- Always answer the request using the given context information and not prior knowledge"+
            "\n- Provide a detailed explanation, but avoid repetitions."+
            "\n- Always answer in norwegian"
        ),
    )
    ,
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Query: {query_str}\n"
            "Answer: "
        ),
    ),
]
text_qa_template =  ChatPromptTemplate(chat_text_qa_msgs)

response_synthesizer = get_response_synthesizer(
    response_mode= "tree_summarize",
    text_qa_template = text_qa_template,
    summary_template= text_qa_template, #definitly in use for response_mode = tree_summarize
    structured_answer_filtering=True, 
    verbose=True,
)

# intialize the LLM engine:
storage = './blobstorage/chatbot/ungnospmtobakk'
index = read_index_from_storage(storage)

query_engine = index.as_query_engine(
    similarity_cutoff=0.7,
    similarity_top_k=10,
    response_synthesizer=response_synthesizer,
)

class Reference(TypedDict):
    name: str
    url: str
    relevance_index: float    

class Feedback(BaseModel):
    grade: Literal ["readable", "not readable"] = Field(description="Decide if an answer is readable or not.")
    feedback: str = Field(description = "Answer is not readable, provide feedback on how to improve it")

    
# Augment the LLM with schema for structured output
evaluator = llm.with_structured_output(Feedback)
    
# Graph state
class State(TypedDict):
    query_engine: BaseQueryEngine
    similarity_cutoff:float
    query: str
    query_short_version: str
    query_summary: str
    response: Response
    response_validity: Literal["valid", "not valid"]
    nodes_with_relevancy : List[NodeWithScore]
    answer: str
    structured_answer: str
    lix_score: float
    lix_category: str
    references: List[Reference]
    feedback: str  # <-- legg til denne
    readable_or_not: Literal["readable", "not readable"]  # legg gjerne til denne også
    
# Function to categorize LIX score
def categorize_lix(lix):
    if lix < 25:
        return "Svært lettlest (for barn)"
    elif 25 <= lix < 35:
        return "Lettlest (enkel litteratur, aviser)"
    elif 35 <= lix < 45:
        return "Middels vanskelig (standard aviser, generell sakprosa)"
    elif 45 <= lix < 55:
        return "Vanskelig (akademiske tekster, offisielle dokumenter)"
    else:
        return "Svært vanskelig (vitenskapelig litteratur)"
    
# Nodes
def llm_call_short_version_generator(state: State):
    """LLM generates a short version for the query"""
    try: 
        msg = llm.invoke(
                f"Give a title in norwegian to the query, ensuring that the 'I' form is preserved: {state['query']}, use only one short sentence"
            )
        return {"query_short_version": msg.content}
    except Exception as e:
        return {"query_short_version": f"Error: {str(e)}"}
    
def llm_call_summary_generator(state: State):
    """LLM generates a summary for the query"""
    try:
        msg = llm.invoke(
                f"Please provide a summary of the user's question in norwegian, ensuring that the 'I' form is preserved : {state['query']}, use only one sentence"
            )
        return {"query_summary": msg.content}
    except Exception as e:
        return {"query_summary": f"Error: {str(e)}"}

def calculate_readability_index(state: State):

    text = state["answer"]
    
    if text :
        # Count words
        words = text.split()
        num_words = len(words)

        # Count sentences (assuming sentences end with '.', '!', or '?')
        num_sentences = len(re.split(r'[.!?]', text)) - 1  # Remove trailing empty splits

        # Count long words (more than 6 letters)
        num_long_words = sum(1 for word in words if len(re.sub(r'[^a-zA-Z]', '', word)) > 6)

        # Calculate LIX
        lix_score = (num_words / num_sentences) + (num_long_words / num_words) * 100
        
        # Get LIX category
        lix_category = categorize_lix(lix_score)
        print(f'\n{lix_score}')
        state['lix_score'] = lix_score
        state['lix_category'] = lix_category

        
    return 

def llm_call_answer(state: State):
    """LLM generate answer"""
    print('LLM generate answer')
    try:
        response_obj = query_engine.query(state['query'])  
        # Write the  answer to the state
        return {"answer":  response_obj.response, "response": response_obj}
    except Exception as e:
        return {"answer": f"Error: {str(e)}"}

def llm_make_answer_more_readable(state: State):
    """LLM make the answer more readable"""
    print(f'LLM make the answer more readable')
    try: 
        msg = llm.invoke(f"This is the answer to improve {state['answer']}, {state['feedback']}")
        return {"answer":  msg.content}
    except Exception as e:
        return {"answer": f"Error: {str(e)}"}

def response_relevancy_evaluator(state: State):
    """Test the nodes from the response for relevancy"""
    if state["response"] is None:
        print ("failed to get the response")
        return {"response_validity": "not valid"}
    else:
        return {"response_validity": "valid"}
    
def references_generator(state: State):
    references : List[Reference] = []
    response = state["response"]
    similarity_cutoff = state["similarity_cutoff"]
    
    # Filter out nodes with score=None
    nodes_with_scores = [node for node in response.source_nodes if node.score is not None]

    for node in nodes_with_scores:
        if node.score >= similarity_cutoff:
            metadata = node.metadata
            #print(f'Metadata: {metadata}')
            text = node.text
            url = metadata.get('url', 'Ingen URL')
            title = metadata.get('title', 'Ingen tittel').lstrip()
            reference: Reference = {
                "name" : title,
                "url" : url,
                "relevance_index" : node.score
            }
            references.append(reference)
        
    return( {"references": references} )
    

def readability_evaluator(state: State):
    """Evaluates the readability of the answer"""
    
    calculate_readability_index(state)

    if state["lix_score"] > 35:
        return {
            "readable_or_not": "not readable",
            "feedback": "Make this text more readable by using shorter sentences, fewer words, and simpler language."
        }
    else:
        return {
            "readable_or_not": "readable",
            "feedback": "No need for improvements"
        }

# Conditional edge function to route back to call for more readable answer
def route_answer(state: State):
    """Route back to answer generator or end based upon feedback from the evaluator"""

    if state["readable_or_not"] == "readable":
        return "Accepted"
    elif state["readable_or_not"] == "not readable":
        return "Rejected + Feedback"
    
# Conditional edge function to route to construction of a structured answer based upon a result from the evaluator
def validate_response(state: State):
    """Route to answer generator or end based upon feedback from the evaluator"""
    
    response = state["response"]
    
    # Filter out nodes with score=None
    nodes_with_scores = [node for node in response.source_nodes if node.score is not None]
    
    if nodes_with_scores:
        for node in nodes_with_scores:
            if node.score >= state["similarity_cutoff"]:
                print(f'found node with score: {node.score}')
                return "Accepted"

    return "Rejected + Feedback"
    
    
def response_builder_node(state: State) -> State:
    print("Starting parallel calls for constructing a structures response")
    return state  # just pass along
    
def aggregator(state: State):
    """Combine answer with structured info into Markdown format"""
    
    combined = f"# Oppsummering av svaret\n\n"
    
    combined += f"## Spørsmålet fra brukeren\n"
    combined += f"{state['query']}\n\n"
    
    combined += f"## Tittel\n"
    combined += f"{state['query_short_version']}\n\n"
    
    combined += f"## Kort sammendrag av spørsmålet\n"
    combined += f"{state['query_summary']}\n\n"
    
    combined += f"## Lettlest svar\n"
    combined += f"{state['answer']}\n\n"
    
    # Optional: include references if you have them
    if "references" in state and state["references"]:
        combined += "## Referanser\n"
        for ref in state["references"]:
            combined += f"- [{ref['name']}]({ref['url']}) (Relevans: {ref['relevance_index']:.2f})\n"
    
    return {"structured_answer": combined}
    
# Build workflow
optimizer_builder = StateGraph(State)

# Add the nodes
optimizer_builder.add_node("llm_call_short_version_generator", llm_call_short_version_generator)
optimizer_builder.add_node("response_relevancy_evaluator", response_relevancy_evaluator)
optimizer_builder.add_node("llm_call_summary_generator", llm_call_summary_generator)
optimizer_builder.add_node("llm_call_answer", llm_call_answer)
optimizer_builder.add_node("readability_evaluator", readability_evaluator)
optimizer_builder.add_node("llm_make_answer_more_readable", llm_make_answer_more_readable)
optimizer_builder.add_node("aggregator", aggregator)
optimizer_builder.add_node("response_builder_node", response_builder_node)
optimizer_builder.add_node("references_generator", references_generator)

# # Add edges to connect nodes
optimizer_builder.add_edge(START, "llm_call_answer")
optimizer_builder.add_edge("llm_call_answer", "response_relevancy_evaluator")

optimizer_builder.add_edge("response_builder_node", "readability_evaluator")
optimizer_builder.add_edge("response_builder_node", "llm_call_short_version_generator")
optimizer_builder.add_edge("response_builder_node", "llm_call_summary_generator")
optimizer_builder.add_edge("response_builder_node", "references_generator")


optimizer_builder.add_edge("llm_make_answer_more_readable", "readability_evaluator")

optimizer_builder.add_edge("llm_call_short_version_generator", "aggregator")
optimizer_builder.add_edge("llm_call_summary_generator", "aggregator")
optimizer_builder.add_edge("references_generator", "aggregator")
optimizer_builder.add_edge("aggregator", END)

optimizer_builder.add_conditional_edges(
    "response_relevancy_evaluator",
    validate_response,
    {  # Name returned by route_answer : Name of next node to visit
        "Accepted": "response_builder_node",
        "Rejected + Feedback": END,
    },
)


optimizer_builder.add_conditional_edges(
    "readability_evaluator",
    route_answer,
    {  # Name returned by route_answer : Name of next node to visit
        "Accepted": "aggregator",
        "Rejected + Feedback": "llm_make_answer_more_readable",
    },
)


# Compile the workflow
optimizer_workflow = optimizer_builder.compile()

#from graph_utils import save_mermaid_diagram
from graph_utils import save_mermaid_diagram
#save_mermaid_diagram(optimizer_workflow.get_graph())

# Invoke
state = optimizer_workflow.invoke({"query_engine": query_engine,
                                   "similarity_cutoff": 0.7, 
                                   "query": "Hei, jeg vaper daglig og har gjort ganske lenge. Jeg begynner å bli stressa og har lyst til å stoppe, men jeg får det ikke til. Jeg har mener at jeg har fått dårligere hud, munnsår, tjukkere ansikt (men har ikke lagt på meg), blir mer sjesken på mat når jeg ikke har vape. Kan munnsår ha noe med vape å gjøre eller ikke? Er mye jeg lurer på i samme spørsmål"})

from IPython.display import Markdown
Markdown(state["structured_answer"])

LLM generate answer
1 text chunks after repacking
found node with score: 0.9007370712205328
Starting parallel calls for constructing a structures response

30.27155727155727


# Oppsummering av svaret

## Spørsmålet fra brukeren
Hei, jeg vaper daglig og har gjort ganske lenge. Jeg begynner å bli stressa og har lyst til å stoppe, men jeg får det ikke til. Jeg har mener at jeg har fått dårligere hud, munnsår, tjukkere ansikt (men har ikke lagt på meg), blir mer sjesken på mat når jeg ikke har vape. Kan munnsår ha noe med vape å gjøre eller ikke? Er mye jeg lurer på i samme spørsmål

## Tittel
"Kan vaping påvirke helsen min?"

## Kort sammendrag av spørsmålet
Hei, jeg lurer på om munnsår kan ha noe med vaping å gjøre, siden jeg har vaper daglig, begynner å bli stressa og ønsker å stoppe, men opplever dårligere hud, munnsår, tjukkere ansikt uten å ha lagt på meg, og mer sult på mat når jeg ikke har vape.

## Lettlest svar
Hei!

Jeg forstår at du føler deg stresset og ønsker å slutte å vape. Det er bra at du tenker på helsen din. Når det gjelder munnsår, tjukkere ansikt og sjenanse på mat, kan det være flere faktorer som spiller inn. Vaping kan påvirke huden din, men det er ikke sikkert at det er den eneste årsaken til hudproblemene dine. Munnsår kan også ha ulike årsaker, men det kan være lurt å være oppmerksom på om det kan være en sammenheng med vaping.

Når det gjelder å slutte å vape, kan det være utfordrende på grunn av avhengighet. Det er viktig å være motivert og ha en plan for å slutte. Du kan oppleve abstinenssymptomer når du slutter, som stress, sjenanse på mat og andre symptomer du nevnte. Disse symptomene kan være midlertidige og vil avta etter hvert som kroppen tilpasser seg.

Det kan være lurt å søke støtte fra helsepersonell eller en rådgiver for å få hjelp til å slutte å vape. De kan gi deg råd og veiledning for å takle abstinenssymptomer og støtte deg gjennom prosessen med å slutte.

Husk at det er aldri for sent å ta vare på helsen din og slutte med vaner som ikke er bra for deg. Lykke til, og ikke nøl med å søke hjelp hvis du trenger det!

Vennlig hilsen,
Din assistent

## Referanser
- [Kan man få kviser av vaping? — Ung.no](https://www.ung.no/oss/1CZBNuIlYCNLm1OAFy2Uts) (Relevans: 0.90)
- [Har sluttet å vape. Har jeg skadet lungene mine? — Ung.no](https://www.ung.no/oss/7cy8QWTOAUDsdYFcrrfuVv) (Relevans: 0.89)
- [Tror jeg er avhengig og er lungene ødelagt av vape? — Ung.no](https://www.ung.no/oss/fXPkeBhneNpvW1NHaMybpM) (Relevans: 0.88)
- [Jeg vil gjerne slutte med vape, synes nå det er teit at jeg begynte? — Ung.no](https://www.ung.no/oss/MowV7UUn50SgMtSE2C54f4) (Relevans: 0.88)
- [Kan man bli sår i halsen og få mer slim av å vape? — Ung.no](https://www.ung.no/oss/xFx6R0VpreHXZthyUnfthX) (Relevans: 0.88)
- [Har vapet og prøvd røyk en liten periode, vil jeg få rynker? — Ung.no](https://www.ung.no/oss/2hER7Y3A5Cqjg4Jhw13swI) (Relevans: 0.88)
- [Kan det å vape litt være skadelig? — Ung.no](https://www.ung.no/oss/17JoCmSdTn9AdtkKfVrtAu) (Relevans: 0.88)
- [Vil lungene mine bli bedre hvis jeg slutter å vape? — Ung.no](https://www.ung.no/oss/q533WEMU35eJVTMbUF5evg) (Relevans: 0.88)
- [Hvorfor har jeg blitt kortpustet av vape? — Ung.no](https://www.ung.no/oss/akHdZQICxt8JjivM40lvv1) (Relevans: 0.88)
- [Kan jeg fortsatt få helseskader hvis jeg slutter helt å vape nå? — Ung.no](https://www.ung.no/oss/WdwvWKVMBI6e9tAqpSnbHM) (Relevans: 0.88)
